In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)  # 设置显示数据的最大列数，防止出现省略号…，导致数据显示不全
pd.set_option('expand_frame_repr', False)  # 当列太多时不自动换行

In [2]:
df1 = pd.DataFrame(np.random.randint(0, 2, (10000, 10)), columns=[f'x{i}' for i in range(10)])

In [3]:
df1.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9
0,0,1,0,0,0,1,1,0,0,1
1,1,1,0,0,1,0,1,1,1,0
2,0,1,0,0,0,0,0,1,1,1
3,0,0,0,1,1,0,0,1,0,1
4,0,1,1,0,1,0,0,0,0,1


In [4]:
df1['y'] = 0

In [5]:
for i in range(10):
    df1.loc[df1[f'x{i}']==1, 'y'] = 1

In [6]:
df1['y'].value_counts()

1    9992
0       8
Name: y, dtype: int64

In [7]:
cols = [f'x{i}' for i in range(10)]
cols.append('y')

In [8]:
df1 = pd.concat((df1, pd.DataFrame(np.zeros((60000, 11), dtype=np.int32), columns=cols)), ignore_index=True)

In [9]:
for i in range(10):
    df2 = pd.DataFrame(np.zeros((5000, 11), dtype=np.int32), columns=cols)
    df2[f'x{i}'] = 1
    df2['y'] = 1
    df1 = pd.concat((df1, df2), ignore_index=True)

In [10]:
df1.shape

(120000, 11)

In [11]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [12]:
model = tf.keras.Sequential()
model.add(Dense(500, activation='sigmoid', input_shape=(10,)))
model.add(Dense(2, activation='softmax'))
adam = Adam(lr=1e-4)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 500)               5500      
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1002      
Total params: 6,502
Trainable params: 6,502
Non-trainable params: 0
_________________________________________________________________


In [13]:
dfVal = df1.values
np.random.shuffle(dfVal)
print(dfVal.shape)
   

(120000, 11)


In [14]:
data = dfVal[:, :-1].astype(np.float32)
label = dfVal[:, -1].astype(np.float32)
label = np.array([tf.keras.utils.to_categorical(y, num_classes=2) for y in label])
print(label.shape)
dataset = tf.data.Dataset.from_tensor_slices((data, label))
dataset = dataset.repeat(1000).batch(5000)                

(120000, 2)


In [17]:
model.fit(dataset, steps_per_epoch=110,epochs=20)

Epoch 1/10
110/110 [==============================] - 4s 40ms/step - loss: 0.2807 - acc: 1.0000
Epoch 2/10
110/110 [==============================] - 5s 42ms/step - loss: 0.2392 - acc: 1.0000
Epoch 3/10
110/110 [==============================] - 5s 46ms/step - loss: 0.2016 - acc: 1.0000
Epoch 4/10
110/110 [==============================] - 5s 41ms/step - loss: 0.1689 - acc: 1.0000
Epoch 5/10
110/110 [==============================] - 4s 38ms/step - loss: 0.1411 - acc: 1.0000
Epoch 6/10
110/110 [==============================] - 5s 42ms/step - loss: 0.1179 - acc: 1.0000
Epoch 7/10
110/110 [==============================] - 5s 41ms/step - loss: 0.0988 - acc: 1.0000
Epoch 8/10
110/110 [==============================] - 4s 40ms/step - loss: 0.0831 - acc: 1.0000
Epoch 9/10
110/110 [==============================] - 5s 43ms/step - loss: 0.0703 - acc: 1.0000
Epoch 10/10
110/110 [==============================] - 4s 39ms/step - loss: 0.0597 - acc: 1.0000


In [16]:
model.predict(np.array([[1]+[0]*9]))

array([[0.3009542, 0.6990458]], dtype=float32)

In [18]:
model.predict(np.array([[1]+[0]*9]))

array([[0.06226559, 0.93773437]], dtype=float32)

In [22]:
model.predict(np.array([[0.9]+[0]*9]))

array([[0.10371412, 0.89628595]], dtype=float32)

In [25]:
model.predict(np.array([[0.5]+[0]*9]))

array([[0.5185505, 0.4814495]], dtype=float32)

In [21]:
model.predict(np.array([[0]+[1]+[0]*8]))

array([[0.06255143, 0.9374485 ]], dtype=float32)

In [20]:
model.predict(np.array([[1]+[1]+[0]*8]))

array([[2.7568123e-04, 9.9972433e-01]], dtype=float32)

In [26]:
model.predict(np.array([[0.5]+[0.5]+[0]*8]))

array([[0.06226441, 0.9377356 ]], dtype=float32)

In [19]:
model.predict(np.array([[0]+[0]*9]))

array([[0.9464888 , 0.05351121]], dtype=float32)